## Woof vs. Meow - 2

### Building an image classification model using CNN along with data augmentation techniques  


## Data

Data can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data  
All you need is the train set  
The recommended folder structure is:  

### Folder structure

```python
data/
    train/
        dogs/ 
            dog001.jpg
            dog002.jpg
            ...
        cats/ 
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/ 
            dog001.jpg
            dog002.jpg
            ...
        cats/ 
            cat001.jpg
            cat002.jpg
            ...
```

### Data loading

In [1]:
##This notebook is built around using tensorflow as the backend for keras
!pip install pillow
!KERAS_BACKEND=tensorflow python -c "from keras import backend"

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Using TensorFlow backend.


In [46]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
import scipy
import pylab as pl
import matplotlib.cm as cm
%matplotlib inline

In [2]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

### Imports

In [3]:
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

Found 20002 images belonging to 2 classes.
Found 4998 images belonging to 2 classes.


## Convolutional Network with data augmentation

### Model architecture definition

In [4]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [5]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Training

In [8]:
nb_epoch = 30
nb_train_samples = 20002
nb_validation_samples = 4998

## Data augmentation for improving the model

By applying random transformation to our train set, we artificially enhance our dataset with new unseen images.  
This will hopefully reduce overfitting and allows better generalization capability for our network.

Example of data augmentation applied to a picture:
![Example of data augmentation applied to a picture](images/data_augmentation.png)

In [11]:
train_datagen_augmented = ImageDataGenerator(
        rescale=1./255,        # normalize pixel values to [0,1]
        shear_range=0.2,       # randomly applies shearing transformation
        zoom_range=0.2,        # randomly applies shearing transformation
        horizontal_flip=True)  # randomly flip the images

# same code as before
train_generator_augmented = train_datagen_augmented.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

Found 20002 images belonging to 2 classes.


In [21]:
model.fit_generator(
        train_generator_augmented,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

Epoch 1/30
20002/20002 [==============================] - 1742s - loss: 0.4735 - acc: 0.8021 - val_loss: 0.4166 - val_acc: 0.8211
Epoch 2/30
20002/20002 [==============================] - 2018s - loss: 0.4578 - acc: 0.8044 - val_loss: 0.4209 - val_acc: 0.8237
Epoch 3/30
20002/20002 [==============================] - 2448s - loss: 0.4553 - acc: 0.8051 - val_loss: 0.4589 - val_acc: 0.7985
Epoch 4/30
20002/20002 [==============================] - 2455s - loss: 0.4615 - acc: 0.8064 - val_loss: 1.2699 - val_acc: 0.6579
Epoch 5/30
20002/20002 [==============================] - 3132s - loss: 0.4581 - acc: 0.8113 - val_loss: 1.0254 - val_acc: 0.7167
Epoch 6/30
20002/20002 [==============================] - 2691s - loss: 0.4513 - acc: 0.8062 - val_loss: 0.4289 - val_acc: 0.8139
Epoch 7/30
20002/20002 [==============================] - 3468s - loss: 0.4471 - acc: 0.8121 - val_loss: 0.4773 - val_acc: 0.7937
Epoch 8/30
20002/20002 [==============================] - 2280s - loss: 0.4525 - acc: 0.81

In [22]:
model.save_weights('models/augmented_30_epochs.h5')

In [12]:
model.load_weights('models/augmented_30_epochs.h5')

### Evaluating on validation set

Computing loss and accuracy :

In [13]:
model.evaluate_generator(validation_generator, nb_validation_samples)

[0.53702074628011753, 0.77991196480976577]

**Due to data augmentation, the accuracy of the model decreased slightly to 78% ** 

This is because our model has now become more robust to scale and rotation changes, and generalizes better on unseen data at the expense of accuracy!